In [1]:
import sys
from os.path import expanduser

adam_home_defined = expanduser("~") + "/adam_home" # / syntax works for Windows, Mac, and Linux
sys.path.append(adam_home_defined) 
#import adam modules after setting adam directory path 
from adam import adam_config
# Set up adam paths
ADAMpaths = adam_config.setPaths.initPaths(adam_home_defined)

#import adam
#from adam import adam_config
from adam import Auth
from adam import ConfigManager
from adam import RestRequests

import webbrowser
import urllib
import os
from shutil import copyfile


Changing adam home path to =  C:\Users\macuser/adam_home


In [2]:
#Attempts to read config that is already set up from an adam_config.json file in adam_home/config directory.
#Feel free to point it somewhere else. If it fails to read that file, it will copy the template from adam.
config_file  = ADAMpaths[3] #adam_config.setPaths.env_config_path
template_file = ADAMpaths[2] # adam_config.setPaths.env_template_path

try:
    f = open(config_file)
    f.close()
except:
    copyfile(template_file, config_file)

config_manager = ConfigManager(config_file)
config = config_manager.get_config()

In [3]:
url = config.get_url()
rest = RestRequests(url)
auth = Auth(rest)

In [4]:
token = config.get_token()

try:
    auth.authenticate(token)
    if auth.get_logged_in():
        print('Welcome, ' + auth.get_user())
    else:
        input("A website will pop up and you will be given options to log in to "
              "retrieve a token. Please copy the token and return here. Press enter to show popup and continue...")
        o = urllib.parse.urlparse(url)
        token_url = "%s://%s/%s" % (o.scheme, o.netloc, 'token.html')  # Yes, seriously. This is how it's done.
        webbrowser.open(token_url)
        
        token = input("Token: ")
        print('\n')
        if auth.authenticate(token):
            config.set_token(auth.get_token())
            config_manager.to_file(config_file)
            print('Updated config file with token for ' + url)
            print('Welcome, ' + auth.get_user())
        else:
            print('Could not authenticate user.')
except RuntimeError as e:
    print('Encountered server error while attempting to authenticate: ' + str(e))

Welcome, emmie.king@gmail.com


In [ ]:
#workspace = input('Workspace project UUID: ')
#config.set_workspace(workspace)
#config_manager.to_file(config_file)